# Ares Demo

Example on running Ares.

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ares import construct

## Config File

For this demo, we'll use a direct dictionary rather than loading a JSON file.

In [ ]:
config = {
    "attacker": {
        "attacks": [
            {
                "type": "evasion",
                "name": "ProjectedGradientDescent",
                "params": {
                    "norm": "inf",
                    "eps": 0.03137254901,
                    "eps_step": 0.00784313725,
                    "num_random_init": 0,
                    "max_iter": 1,
                    "targeted": False,
                    "batch_size": 1,
                    "verbose": False
                }
            }
        ]
    },
    "defender": {
        "probabilities": [0.3, 0.7],
        "models": [
            {
                "file": "./data/models/resnet.py",
                "name": "resnet18_nat",
                "params": {
                    "num_classes": 10
                },
                "checkpoint": "./data/state_dicts/resnet18_nat.pth"
            },
            {
                "file": "./data/models/resnet.py",
                "name": "resnet18_adv",
                "params": {
                    "num_classes": 10
                },
                "checkpoint": "./data/state_dicts/resnet18_adv.pth"
            }
        ]
    },
    "scenario": {
        "threat_model": "whitebox",
        "num_episodes": 50,
        "max_rounds": 50,
        "dataset": {
            "name": "cifar10",
            "input_shape": [3, 32, 32],
            "num_classes": 10,
            "clip_values": [0, 1],
            "random_noise": True,
            "dataroot": "./data"
        }
    }
}

## Environment

Create the attacker, defender, and scenario first. Then initialize the Gym environment using the three components.

In [ ]:
# create environment
env = construct(config)
env

## Run Experiment

Run the simulation for the number of episodes specified in the execution scenario. Record the number of rounds until the attacker wins for each trial.

In [ ]:
episode_rewards = []
for episode in range(env.scenario.num_episodes):
    print(f'=== Episode {episode + 1} ===')

    # initialize environment
    observation = env.reset()
    x = observation['x']
    y = observation['y']
    done = False

    # run simulation
    while not done:
        action = {
            'x': x,
            'y': y,
        }
        observation, reward, done, info = env.step(action)
        x = observation['x_adv']
        y_pred = observation['y_pred']
        winner = observation['winner']
        step_count = info['step_count']

        print(f'Step {step_count:2}: ({y[0]} | {y_pred[0]})')

    print(f'Game end: {winner} wins after {reward} rounds')
    episode_rewards.append(reward)

## Statistics

Statistics and visualizations for the experiment.

In [ ]:
print(episode_rewards)
print(f'min: {np.min(episode_rewards)}, max: {np.max(episode_rewards)}')
mean = np.mean(episode_rewards)
stddev = np.std(episode_rewards)
median = np.median(episode_rewards)
print(f'mean: {mean}, stddev: {stddev:.3f}, median: {median}')

plt.hist(episode_rewards)
plt.title('Distribution of rounds until attacker win')
plt.show()